# Rental price analysis of german cities
__Creating and Altering Dataframes__ \
__by Maximilian Hilbert__

In [1]:
import re
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from numpy import savetxt

__For later use__

Source for inflation data: https://de.statista.com/statistik/daten/studie/1046/umfrage/inflationsrate-veraenderung-des-verbraucherpreisindexes-zum-vorjahr/ \
Source for ratio of inflation caused by rentals: https://service.destatis.de/Voronoi/PreisKaleidoskop.svg

In [2]:
def inflation_adjustment(inflation,means_raw):
    ret_lst=[]
    for idx in range(len(means_raw)):
        z=1
        value=means_raw[idx]
        for i in range(idx):
            z*=1/inflation[i]
        adjusted_value=z*value
        ret_lst.append(adjusted_value)
    return ret_lst

In [3]:
#inflation adjustment and correction for rental housing
correction_factor=0.8
infl_raw=[1.1,2.1,2,1.4,1,0.5,0.5,1.5] #in %
infl_lst_full=[1.011,1.021,1.02,1.014,1.01,1.005,1.005,1.015] #final factors
infl_lst_part=[1+(x*correction_factor/100) for x in infl_raw]

infl_lst_2002_2017=[1.013,1.011,1.017,1.015,1.016,1.023,1.026,1.003,1.011,1.021,1.02,1.014,1.01,1.005,1.005,1.015]

__Part 1 - Alter .html files for each city by http://www.wohnung.com/mietpreise__

__For easier use, we clean off all whitespace characters like /n, /t and /s out of each .html-file__

In [4]:
files=os.listdir("data\\raw_files")

In [5]:
def cleaning_html():
    #for each .html file in the subdirectory
    for i in tqdm(range(len(files))):
        filename=files[i]
        #read data out of file
        with open("data\\raw_files\\"+filename, "r", encoding="utf-8") as file:
            text=file.read()
            file.close()
        #delete all whitespaces
        new_text=re.sub("\s+", "", text)
        #save the "new" file
        with open("data\\raw_files\\"+filename, "w", encoding="utf-8") as file:
            file.write(new_text)
            file.close()
    return "Done."

In [6]:
cleaning_html()

100%|██████████| 2469/2469 [00:02<00:00, 1077.29it/s]


'Done.'

__Now we are interested in looking at the content of each .html-file and we notice, that the data we want to get is contained in an java-script linechart, which we can extract by figuring out a suitable regex__

In [7]:
def create_dict_data():
    values={}
    #for each .html file in the subdirectory read contents
    for i in tqdm(range(len(files))):
        filename=files[i]
        file=open("data\\raw_files\\"+filename, "r", encoding="utf-8")
        text=file.read()
        try:
        #find all values that are between the "id=rentprice-development-linechart.js" and the end of the script
        #and get the first appearing value
            raw=re.findall("(?=id\=\"rentprice\-development\-linechart\-chartjs)(.*?)(?<=\<\/script\>)", text)[0]
        except IndexError:
        #if there is no second appearing entry an IndexError is being rased, if so, ignore that and progress 
            pass
        try:
            #in the raw string find all values that are between "data:[" and "]" that are seperated by a ","
            #and get the first appearing value
            data=re.findall("(?=data\:\[)([\S]*?)(?<=\,\])", raw)[0]
            #find all numerical values between "." (important, because there are some int values e.g. "9"
            # and floats like "9.89")
            data=re.findall("[0-9]+\.[0-9]+", data)
            #crop .html ending out of filename for convenience
            cityname=re.findall("(.*)(?=.html)",filename)[0]
            #save the values in a dictionary for easier use later
            values[cityname]=[float(x) for x in data]
        except TypeError:
            #Pass a specific error that occurs because of edge-cases
            pass
    return values

In [8]:
def create_dict_data():
    values={}
    for i in tqdm(range(len(files))):
        filename=files[i]
        file=open("data\\raw_files\\"+filename, "r", encoding="utf-8")
        text=file.read()
        try:
            raw=re.findall("(?=id\=\"rentprice\-development\-linechart\-chartjs)(.*?)(?<=\<\/script\>)", text)[0]
        except IndexError:
            pass
        try:
            data=re.findall("(?=data\:\[)([\S]*?)(?<=\,\])", raw)[0]
            data=re.findall("[0-9]+\.[0-9]+", data)
            #crop .html ending out of filename
            cityname=re.findall("(.*)(?=.html)",filename)[0]
            values[cityname]=[float(x) for x in data]
        except TypeError:
            pass
    return values

def changing_order(dict):
    for element in dict.values():
        while len(element)<16:
            element.insert(0, np.nan)
    return dict
    
dict_data=create_dict_data()
df_raw=pd.DataFrame.from_dict(changing_order(dict_data), orient="index")

100%|██████████| 2469/2469 [00:44<00:00, 55.93it/s]


__Save dictionary in RAM__

In [9]:
dict_data=create_dict_data()

100%|██████████| 2469/2469 [00:01<00:00, 2001.15it/s]


__Because there is a lack in data for some cities that have started collecting data later than others, we have to be sure, that the values are sorted in the correct order and values in the past (that are missing) will get filled with dummy values like NaN__

In [10]:
def changing_order(dict):
    #iterate through the dictionary of values
    for element in dict.values():
        #fill each dictionary entry until it has 16 values (reaching from 2002 to 2017)
        while len(element)<16:
            #insert NaNs if a value is equal to 0
            element.insert(0, np.nan)
    return dict

__Save cleaned data into pandas-dataframe__

In [11]:
df_raw=pd.DataFrame.from_dict(changing_order(dict_data), orient="index")

__Now we rename columns for convenience and make corrections of obviously wrong data__

In [12]:
def rename(dataframe):
    years=[x for x in range(2002, 2018)]
    #Some values are obviously wrong and equal to 0, we correct them by replacing them to NaN that averages and 
    #standard devs. dont get affected
    dataframe[dataframe.columns]=dataframe[dataframe.columns].replace(0.0, np.nan)
    #Renaming columns to corresponding years (would have been possible to do that by getting the labels out of the
    #java-script plot, but this way is easier)
    dataframe=dataframe.rename({0: years[0], 1: years[1], 0: years[0], 1: years[1],2: years[2], 3: years[3],4: years[4], 5: years[5],
                6: years[6], 7: years[7],8: years[8], 9: years[9],0: years[0], 10: years[10],11: years[11], 12: years[12],
                13: years[13], 14: years[14],15: years[15]}, axis=1)
    return dataframe

__Perform the first calulcations on the data in order to get the averages, standard dev. and the number of available datapoints per year__

In [13]:
def calc(dataframe):
    dataframe=dataframe.T
    dataframe["Mean in $\\mathrm{\\frac{EUR}{m^2}}$"]=dataframe.mean(skipna=True, axis=1)
    dataframe["SD in $\\mathrm{\\frac{EUR}{m^2}}$"]=dataframe.std(skipna=True, axis=1)
    dataframe["Historical data points"]=dataframe.count(axis=1)
    return dataframe

__Rename and calculate in one step__

In [14]:
df_calc=calc(rename(df_raw))

__Inflation adjustment__

In [15]:
df_copy=df_calc.copy()

In [16]:
df_copy["Mean in $\\mathrm{\\frac{EUR}{m^2}}$"]=inflation_adjustment(infl_lst_2002_2017,df_copy["Mean in $\\mathrm{\\frac{EUR}{m^2}}$"].values)

In [17]:
df_copy["SD in $\\mathrm{\\frac{EUR}{m^2}}$"]=inflation_adjustment(infl_lst_2002_2017,df_copy["SD in $\\mathrm{\\frac{EUR}{m^2}}$"].values)

In [18]:
df_copy=df_copy.round(2)

In [19]:
df_copy["..."]=["..." for x in range(len(df_copy))]

In [20]:
df_copy

,aach,aachen,aalen,aarbergen,abensberg,absberg,abstatt,achim,achtrup,adelschlag,...,zuelpich,zusmarshausen,zweibruecken,zwenkau,zwickau,zwingenberg,Mean in $\mathrm{\frac{EUR}{m^2}}$,SD in $\mathrm{\frac{EUR}{m^2}}$,Historical data points,...
2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.48,0.74,8,...
2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.27,0.77,11,...
2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.66,0.95,15,...
2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.27,0.74,15,...
2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.22,0.67,17,...
2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.93,0.79,18,...
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.91,1.00,135,...
2009,6.68,NaN,5.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.58,NaN,NaN,NaN,5.55,1.29,410,...
2010,5.14,6.95,6.10,5.20,4.53,4.72,7.11,5.64,5.45,NaN,...,5.27,5.55,4.88,5.02,4.80,7.62,5.27,1.28,2254,...
2011,5.68,7.19,6.15,5.46,6.05,4.51,6.56,5.89,4.99,NaN,...,5.35,5.72,4.93,4.97,4.84,7.50,5.30,1.25,2382,...


In [21]:
tex_table=df_copy[["aach", "aachen", "...", "Mean in $\\mathrm{\\frac{EUR}{m^2}}$", "SD in $\\mathrm{\\frac{EUR}{m^2}}$", "Historical data points"]]

In [22]:
tex_table

,aach,aachen,...,Mean in $\mathrm{\frac{EUR}{m^2}}$,SD in $\mathrm{\frac{EUR}{m^2}}$,Historical data points
2002,NaN,NaN,...,5.48,0.74,8
2003,NaN,NaN,...,5.27,0.77,11
2004,NaN,NaN,...,5.66,0.95,15
2005,NaN,NaN,...,5.27,0.74,15
2006,NaN,NaN,...,5.22,0.67,17
2007,NaN,NaN,...,4.93,0.79,18
2008,NaN,NaN,...,4.91,1.00,135
2009,6.68,NaN,...,5.55,1.29,410
2010,5.14,6.95,...,5.27,1.28,2254
2011,5.68,7.19,...,5.30,1.25,2382


In [23]:
with open('..\\paper\\Table_raw.tex','w') as tf:
    tf.write(tex_table.to_latex(escape=False))

__Keep raw dataframe for later use, without any calculations in it__

In [24]:
df_raw=rename(df_raw)

In [25]:
df_raw

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
aach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.68,5.14,5.68,5.97,5.09,6.53,6.37,6.69,7.04
aachen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.95,7.19,7.63,7.92,8.16,8.30,8.64,8.87
aalen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.90,6.10,6.15,6.39,6.73,7.12,7.62,8.38,8.66
aarbergen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.20,5.46,5.17,4.97,5.13,5.17,5.60,5.71
abensberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,6.05,5.81,5.96,5.93,6.87,8.01,7.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zusmarshausen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.55,5.72,6.00,6.33,6.17,6.08,7.51,7.68
zweibruecken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.58,4.88,4.93,5.10,5.07,5.14,5.27,5.39,5.71
zwenkau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.02,4.97,5.03,5.19,5.40,5.35,5.59,6.29
zwickau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.80,4.84,4.93,4.99,5.09,5.10,5.23,5.43


__Part 2 - Creating dictionary of states-cities__

__List all manually downloaded .txt files in subdirectory__

In [26]:
states=os.listdir("data\\cities_by_state")

__Now we want to achieve a dictionary that maps for a given city the corresponding state it is matching to__

In [27]:
def create_states_cities_dict():
    dict={}
    for state in states:
        #open each .txt file and state read contents
        with open("data\\cities_by_state\\"+state+"\cities.txt","r", encoding="utf-8") as file:
            text=file.read()
            #regex for finding all names of cities after a "\n", the difficulty is to find different styles of writing
            # for instance Halle (Saale) or  Bad Waldsee. I figured thetr is a maximum of 3 repetitions of "words" and 
            # whitespaces and after each city name there are numbers that shouldn't get matched
            cities=re.findall("\n(\S+\s*\S*\s*\S*\s*\S*)(?=\d{1,2}\,)", text)
            #filter remaining digits, this is necessary because when there are coming 2 digits after a city name one
            #of that is matched
            cities=[re.sub("\d","",element) for element in cities]
            #convert all letters to lowercase and remove whitespaces and german umlauts to achieve a better consistency
            cities=[x.lower() for x in cities]
            cities=[re.sub(" ","",string) for string in cities]
            cities=[re.sub("Ü|ü","ue",string) for string in cities]
            cities=[re.sub("Ä|ä","ae",string) for string in cities]
            cities=[re.sub("Ö|ö","oe",string) for string in cities]
            cities=[re.sub("ß","ss",string) for string in cities]
            dict[state]=cities
    return dict

__Get dictionary and save it__

In [28]:
dict_state_to_city=create_states_cities_dict()

In [29]:
dict_state_to_city

{'Baden-Wuerttemberg': ['aalen',
  'abstatt',
  'achberg',
  'achstetten',
  'adelberg',
  'adelmannsfelden',
  'adelsheim',
  'affalterbach',
  'aglasterhausen',
  'aichelberg',
  'aichhalden',
  'aichstetten',
  'aichtal',
  'aichwald',
  'aidlingen',
  'aitern',
  'aitrach',
  'albbruck',
  'albershausen',
  'albstadt',
  'aldingen',
  'alfdorf',
  'allensbach',
  'alleshausen',
  'allmannsweiler',
  'alpirsbach',
  'altbach',
  'altensteig',
  'altheim',
  'althengstett',
  'althuette',
  'altlussheim',
  'altshausen',
  'ammerbuch',
  'amstetten',
  'amtzell',
  'angelbachtal',
  'appenweier',
  'argenbuehl',
  'asbach',
  'aspach',
  'asperg',
  'assamstadt',
  'asselfingen',
  'attenweiler',
  'auenwald',
  'auggen',
  'aulendorf',
  'backnang',
  'badbellingen',
  'badditzenbach',
  'badduerrheim',
  'badfriedrichshall',
  'badherrenalb',
  'badkrozingen',
  'badliebenzell',
  'badmergentheim',
  'badpeterstal-griesbach',
  'badrappenau',
  'badrippoldsau-schapbach',
  'badschu

__Insert corresponding states into the dataframe in new column__

In [30]:
def match_cities_to_states(dataframe):
    #For each entry in the dataframe
    for city in dataframe.index.values:
        #For every key in the dict
        for key in dict_state_to_city.keys():
            #And every element per key
            for element in dict_state_to_city[key]:
                #If the element (value of a cetain key) is equal to the city name
                if element==city:
                    #get the index of this city entry in the dataframe
                    idx=dataframe[dataframe.index==city].index
                    #set on the index "idx" and the column "state" the key
                    dataframe.loc[idx,"state"]=key
                else:
                    pass
    return dataframe

In [31]:
df_raw=match_cities_to_states(df_raw)

In [32]:
df_raw.head()

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,state
aach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.68,5.14,5.68,5.97,5.09,6.53,6.37,6.69,7.04,Rheinland-Pfalz
aachen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.95,7.19,7.63,7.92,8.16,8.30,8.64,8.87,Nordrhein-Westfalen
aalen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.90,6.10,6.15,6.39,6.73,7.12,7.62,8.38,8.66,Baden-Wuerttemberg
aarbergen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.20,5.46,5.17,4.97,5.13,5.17,5.60,5.71,Hessen
abensberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,6.05,5.81,5.96,5.93,6.87,8.01,7.90,Bayern


In [33]:
df_raw.astype({2010: 'float32',2017: 'float32'})

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,state
aach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.68,5.14,5.68,5.97,5.09,6.53,6.37,6.69,7.04,Rheinland-Pfalz
aachen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.95,7.19,7.63,7.92,8.16,8.30,8.64,8.87,Nordrhein-Westfalen
aalen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.90,6.10,6.15,6.39,6.73,7.12,7.62,8.38,8.66,Baden-Wuerttemberg
aarbergen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.20,5.46,5.17,4.97,5.13,5.17,5.60,5.71,Hessen
abensberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,6.05,5.81,5.96,5.93,6.87,8.01,7.90,Bayern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zusmarshausen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.55,5.72,6.00,6.33,6.17,6.08,7.51,7.68,Bayern
zweibruecken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.58,4.88,4.93,5.10,5.07,5.14,5.27,5.39,5.71,Rheinland-Pfalz
zwenkau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.02,4.97,5.03,5.19,5.40,5.35,5.59,6.29,Sachsen
zwickau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.80,4.84,4.93,4.99,5.09,5.10,5.23,5.43,Sachsen


In [34]:
df_raw

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,state
aach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.68,5.14,5.68,5.97,5.09,6.53,6.37,6.69,7.04,Rheinland-Pfalz
aachen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.95,7.19,7.63,7.92,8.16,8.30,8.64,8.87,Nordrhein-Westfalen
aalen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.90,6.10,6.15,6.39,6.73,7.12,7.62,8.38,8.66,Baden-Wuerttemberg
aarbergen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.20,5.46,5.17,4.97,5.13,5.17,5.60,5.71,Hessen
abensberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,6.05,5.81,5.96,5.93,6.87,8.01,7.90,Bayern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zusmarshausen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.55,5.72,6.00,6.33,6.17,6.08,7.51,7.68,Bayern
zweibruecken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.58,4.88,4.93,5.10,5.07,5.14,5.27,5.39,5.71,Rheinland-Pfalz
zwenkau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.02,4.97,5.03,5.19,5.40,5.35,5.59,6.29,Sachsen
zwickau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.80,4.84,4.93,4.99,5.09,5.10,5.23,5.43,Sachsen


__Now we want to get a grouped dataframe that contains means and standard deviations per state for the rental prices__

In [35]:
df_grouped=df_raw.groupby("state").mean()

In [36]:
df_grouped.head()

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
state,,,,,,,,,,,,,,,,
Baden-Wuerttemberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.759643,6.612102,6.752458,6.846000,7.060615,7.221397,7.507043,7.969811,8.456354
Bayern,NaN,NaN,NaN,NaN,NaN,NaN,4.6,5.922059,6.340102,6.434101,6.655681,6.807259,7.004533,7.284161,7.755324,8.169422
Berlin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.430000,6.980000,7.260000,7.700000,8.270000,8.860000,9.110000,9.830000,10.880000
Brandenburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.736667,6.031250,6.136829,6.513500,6.556829,6.520500,6.699268,7.252439
Bremen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.490000,5.405000,5.650000,5.935000,6.120000,6.230000,6.285000,6.705000,7.135000


In [37]:
df_raw

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,state
aach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.68,5.14,5.68,5.97,5.09,6.53,6.37,6.69,7.04,Rheinland-Pfalz
aachen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.95,7.19,7.63,7.92,8.16,8.30,8.64,8.87,Nordrhein-Westfalen
aalen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.90,6.10,6.15,6.39,6.73,7.12,7.62,8.38,8.66,Baden-Wuerttemberg
aarbergen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.20,5.46,5.17,4.97,5.13,5.17,5.60,5.71,Hessen
abensberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.53,6.05,5.81,5.96,5.93,6.87,8.01,7.90,Bayern
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zusmarshausen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.55,5.72,6.00,6.33,6.17,6.08,7.51,7.68,Bayern
zweibruecken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.58,4.88,4.93,5.10,5.07,5.14,5.27,5.39,5.71,Rheinland-Pfalz
zwenkau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.02,4.97,5.03,5.19,5.40,5.35,5.59,6.29,Sachsen
zwickau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.80,4.84,4.93,4.99,5.09,5.10,5.23,5.43,Sachsen


__Now we are interested in the standard dev. of states containing certain city data. We need that as errorbars for our plots of state data later on. Of course all year to year comparisons should be inflation adjusted.__

In [38]:
df_grouped

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
state,,,,,,,,,,,,,,,,
Baden-Wuerttemberg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.759643,6.612102,6.752458,6.846000,7.060615,7.221397,7.507043,7.969811,8.456354
Bayern,NaN,NaN,NaN,NaN,NaN,NaN,4.600000,5.922059,6.340102,6.434101,6.655681,6.807259,7.004533,7.284161,7.755324,8.169422
Berlin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.430000,6.980000,7.260000,7.700000,8.270000,8.860000,9.110000,9.830000,10.880000
Brandenburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.736667,6.031250,6.136829,6.513500,6.556829,6.520500,6.699268,7.252439
Bremen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.490000,5.405000,5.650000,5.935000,6.120000,6.230000,6.285000,6.705000,7.135000
Hamburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.600000,9.910000,10.320000,10.560000,10.770000,10.770000,11.170000,11.870000
Hessen,NaN,NaN,NaN,NaN,NaN,NaN,4.660000,5.433750,6.073868,6.198645,6.368037,6.463178,6.631495,6.794554,7.041651,7.492465
Mecklenburg-Vorpommern,NaN,NaN,NaN,NaN,NaN,4.360000,7.140000,5.995000,5.522083,5.649750,5.686923,5.740750,5.714000,6.001500,6.222500,6.501250
Niedersachsen,5.481667,5.343333,5.791538,5.493077,5.516667,5.352667,5.500696,5.347500,5.338274,5.412466,5.510959,5.668919,5.846606,6.073333,6.375822,6.542978


In [39]:
mean_infl_per_year_full=np.mean(infl_lst_full)
mean_infl_per_year_part=np.mean(infl_lst_part)
#Sort by means for 2010
df_gr=df_grouped.drop(df_grouped.columns.difference(["index","state",2010,2017]), axis=1)
df_gr.sort_values(by=[2010], ascending=True, inplace=True)
#Create new df where we calculate calculate stds for each state and save it to the new column "std_2010"
df_std_2010=df_raw.groupby("state").agg({2010:np.std}).rename({2010:"std_2010"},axis=1)

#Same for 2017 data
df_gr=df_grouped.drop(df_grouped.columns.difference(["state",2010,2017]), axis=1)
df_gr.sort_values(by=[2017], ascending=True, inplace=True)
df_std_2017=df_raw.groupby("state").agg({2017:np.std}).rename({2017:"std_2017"},axis=1)

#Use mean inflation rate to get inflation adjustment only for 2017
df_std_2017["std_2017_part"]=df_std_2017["std_2017"].apply(lambda x: x/mean_infl_per_year_part**7)
df_std_2017["std_2017"]=df_std_2017["std_2017"].apply(lambda x: x/mean_infl_per_year_full**7)

df_std_2017["mean_2017_part"]=df_gr[2017].apply(lambda x: x/mean_infl_per_year_part**7)
df_std_2017["mean_2017_full"]=df_gr[2017].apply(lambda x: x/mean_infl_per_year_full**7)

# #Merge to 2010/2017 dataframes and generate df_grouped 
df_temp=pd.merge(df_gr,df_std_2010, on="state", how="inner")
df_grouped=pd.merge(df_temp,df_std_2017, on="state", how="inner")

In [40]:
df_grouped=df_grouped.round(2)

In [41]:
df_grouped

,2010,2017,std_2010,std_2017,std_2017_part,mean_2017_part,mean_2017_full
state,,,,,,,
Sachsen-Anhalt,4.75,5.42,0.96,0.59,0.60,5.05,4.96
Sachsen,4.80,5.56,0.76,0.73,0.75,5.18,5.09
Thueringen,5.30,6.30,0.80,2.94,2.99,5.87,5.77
Saarland,5.29,6.34,0.65,0.65,0.66,5.91,5.81
Mecklenburg-Vorpommern,5.52,6.50,0.83,1.38,1.40,6.06,5.95
Nordrhein-Westfalen,5.50,6.52,0.79,0.94,0.95,6.08,5.97
Niedersachsen,5.34,6.54,1.00,1.24,1.26,6.10,5.99
Rheinland-Pfalz,5.65,6.76,1.05,1.31,1.33,6.30,6.20
Bremen,5.40,7.14,1.27,1.82,1.85,6.65,6.54


__Now we save the dataframes to use it in another file and provide them as sources__

In [42]:
df_grouped.to_csv("data\\csv\\grouped.csv")
df_raw.to_csv("data\\csv\\raw.csv")
df_calc.to_csv("data\\csv\\calc_raw.csv")

__Getting inflation data from statista and calculate adjustment for each year in the dataframe__

In [43]:
means_raw=df_calc["Mean in $\\mathrm{\\frac{EUR}{m^2}}$"][8:].values
stds_raw=df_calc["SD in $\\mathrm{\\frac{EUR}{m^2}}$"][8:].values

__Save inflation adjustment for averages and standard devs.__

In [44]:
infl_adjusted_means_full=inflation_adjustment(infl_lst_full,means_raw)
infl_adjusted_std_full=inflation_adjustment(infl_lst_full,stds_raw)

In [45]:
savetxt('data\\csv\\infl_adjusted_means_full.csv', infl_adjusted_means_full, delimiter=',')
savetxt('data\\csv\\infl_adjusted_std_full.csv', infl_adjusted_std_full, delimiter=',')
savetxt('data\\csv\\infl_lst_full.csv', infl_lst_full, delimiter=',')
savetxt('data\\csv\\means_raw.csv', means_raw, delimiter=',')

__Additional table for the paper__

In [47]:
count_17=pd.DataFrame(df_raw.groupby(["state"]).count()[2017])
count_10=pd.DataFrame(df_raw.groupby(["state"]).count()[2010])
df_count=pd.merge(count_17, count_10,on=["state"], how="inner")

In [48]:
df_count=df_count.rename({2017: "data points 2017", 2010: "data points 2010"}, axis=1)

In [49]:
with open('..\\paper\\Table_bavarian_cities.tex','w') as tf:
    tf.write(df_count.to_latex(escape=False))

In [50]:
df_calc

,aach,aachen,aalen,aarbergen,abensberg,absberg,abstatt,achim,achtrup,adelschlag,...,zschorlau,zuelpich,zusmarshausen,zweibruecken,zwenkau,zwickau,zwingenberg,Mean in $\mathrm{\frac{EUR}{m^2}}$,SD in $\mathrm{\frac{EUR}{m^2}}$,Historical data points
2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.481667,0.739063,8
2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.343333,0.782588,11
2004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.791538,0.974315,15
2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.493077,0.768229,15
2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.516667,0.704847,17
2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.290625,0.850665,18
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.390226,1.103880,135
2009,6.68,NaN,5.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4.58,NaN,NaN,NaN,6.253333,1.458711,410
2010,5.14,6.95,6.10,5.20,4.53,4.72,7.11,5.64,5.45,NaN,...,4.11,5.27,5.55,4.88,5.02,4.80,7.62,5.964045,1.448575,2254
2011,5.68,7.19,6.15,5.46,6.05,4.51,6.56,5.89,4.99,NaN,...,5.17,5.35,5.72,4.93,4.97,4.84,7.50,6.060828,1.424802,2382
